In [18]:
import pandas as pd  # Librería para la manipulación y análisis de datos
import numpy as np  # Librería para operaciones matemáticas y manejo de arrays
from sklearn.model_selection import train_test_split  # Función para dividir el dataset en entrenamiento y prueba
from sklearn.preprocessing import StandardScaler  # Clase para normalizar las características
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # Métricas de evaluación del modelo
from xgboost import XGBRegressor  # Modelo de regresión basado en Gradient Boosting
from google.colab import files  # Librería para cargar archivos en Google Colab

# Solicitar al usuario que cargue un archivo CSV desde su máquina local
uploaded = files.upload()

# Obtener el nombre del archivo cargado y leerlo en un DataFrame
filename = list(uploaded.keys())[0]
data = pd.read_csv(filename)

# Mostrar una vista previa del conjunto de datos
print("\nVista previa del dataset:")
print(data.head())
print("\nColumnas disponibles:", list(data.columns))

# Definir el nombre de la columna que contiene la variable objetivo
target_column = 'Price'

# Verificar si la columna objetivo existe en el DataFrame
if target_column not in data.columns:
    print(f"\nAdvertencia: La columna '{target_column}' no se encontró. Verifica el nombre correcto.")
else:
    # Eliminar registros nulos para evitar errores durante el entrenamiento
    data = data.dropna()

    # Separar las características (X) y la variable objetivo (y)
    X = data.drop([target_column], axis=1)  # Eliminar la columna 'price' de las características
    y = data[target_column]  # Asignar la columna 'price' como variable objetivo

    # Convertir variables categóricas en variables dummy (codificación one-hot)
    X = pd.get_dummies(X, drop_first=True)

    # Dividir el conjunto de datos en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Escalar las características para que tengan media 0 y desviación estándar 1
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)  # Ajustar y transformar los datos de entrenamiento
    X_test_scaled = scaler.transform(X_test)  # Transformar los datos de prueba usando el mismo escalador

    # Entrenar el modelo XGBoost con hiperparámetros ajustados
    model = XGBRegressor(
        n_estimators=300,        # Número de árboles en el modelo
        learning_rate=0.1,       # Tasa de aprendizaje para reducir el impacto de cada árbol
        max_depth=5,             # Profundidad máxima de los árboles
        objective='reg:squarederror',  # Función objetivo para regresión
        random_state=42          # Semilla para resultados reproducibles
    )
    model.fit(X_train_scaled, y_train)  # Entrenar el modelo con los datos de entrenamiento

    # Realizar predicciones sobre los datos de prueba
    y_pred = model.predict(X_test_scaled)

    # Calcular las métricas de evaluación
    mae = mean_absolute_error(y_test, y_pred)  # Error absoluto medio
    mse = mean_squared_error(y_test, y_pred)  # Error cuadrático medio
    rmse = np.sqrt(mse)  # Raíz del error cuadrático medio
    r2 = r2_score(y_test, y_pred)  # Coeficiente de determinación (R^2)

    # Imprimir los resultados de las métricas
    print("\nEvaluación del modelo XGBoost:")
    print(f"MAE: {mae:.4f}")  # Interpretación: Promedio de error en la predicción del precio
    print(f"MSE: {mse:.4f}")  # Interpretación: Penaliza los errores elevados más que el MAE
    print(f"RMSE: {rmse:.4f}")  # Interpretación: Valor en la misma escala de los precios predichos
    print(f"R^2: {r2:.4f}")  # Interpretación: Qué tan bien el modelo explica la variabilidad de los datos


Saving clothes_price_prediction_dat_3.csv to clothes_price_prediction_dat_3 (13).csv

Vista previa del dataset:
          Brand Category  Color Size Material  Price
0   New Balance    Dress  White   XS    Nylon    182
1   New Balance    Jeans  Black   XS     Silk     57
2  Under Armour    Dress    Red    M     Wool    127
3          Nike    Shoes  Green    M   Cotton     77
4        Adidas  Sweater  White    M    Nylon    113

Columnas disponibles: ['Brand', 'Category', 'Color', 'Size', 'Material', 'Price']

Evaluación del modelo XGBoost:
MAE: 51.4378
MSE: 3884.6299
RMSE: 62.3268
R^2: -0.3256
